<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-loading" data-toc-modified-id="Data-loading-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data loading</a></span></li><li><span><a href="#Utils" data-toc-modified-id="Utils-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Utils</a></span></li><li><span><a href="#Data-processing" data-toc-modified-id="Data-processing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data processing</a></span></li><li><span><a href="#Testing-the-strategy" data-toc-modified-id="Testing-the-strategy-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Testing the strategy</a></span><ul class="toc-item"><li><span><a href="#Naive-strategy" data-toc-modified-id="Naive-strategy-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Naive strategy</a></span></li><li><span><a href="#Better-strategy" data-toc-modified-id="Better-strategy-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Better strategy</a></span></li><li><span><a href="#Best-strategy" data-toc-modified-id="Best-strategy-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Best strategy</a></span></li></ul></li></ul></div>

# Team Novak


In [40]:
# Importing libraries

import numpy as np
import pickle
import pandas as pd
from nltk import ngrams
from aux_functions import *
from collections import Counter

## Data loading



In [43]:
# Historical data

df = pd.read_pickle('../data/dataframe.p')

# Prompts and shots
all_shots = pickle.load(open("../data/allowed_shots.p", "rb"))
poss_prompts = pickle.load(open("../data/possible_prompts.p" , "rb"))

print('Allowed shots: ', all_shots, '\n')
print('Possible prompts: ', poss_prompts)
df.head()

Allowed shots:  ['4', '4+', '5', '6', '6+', 'X', 'b', 'b1', 'b17', 'b18', 'b19', 'b2', 'b27', 'b28', 'b29', 'b3', 'b37', 'b38', 'f', 'f+1', 'f+3', 'f1', 'f18', 'f2', 'f27', 'f28', 'f29', 'f3', 'f37', 'f38', 'r2', 's', 's1', 's2', 's28', 's3', 'v1', 'v3', 'z1', 'z3'] 

Possible prompts:  ['f3', 'f1', 'b2', 'b1', 'X', 'f2', 'b3', '4', '6', '5', '*', 'b28', 'f28', 'b27', 's2', 'f+3', 's3', 'n@', 'f+1', 'b18', 'd@', 'f38', 'b29', 's1', 'b38', 'f37', '#', 'w@', 'f27', 'f29', 'n#', 'd#', 'f', 'b', 's28', 'b37', 'b17', 'f1*', 'b19', 'b39', 'w#', 'r2', 'f18', 'f39', 'z3', 'v1', 'v3', 'f#', 's27', '4+', '6+', 'z1', 'b1*', 'f3*', 'm2', 'f17', 'r3', 's', 'r1', 's29', 's38', 'b+1', 'z2', 'o3', 'c6', 'v2', 'c4', 'b1w@', 'b#', 'b1n@', 'b+3', 'f19', 'b3*', 'f+2', 'f1w@', 'b3w@', 's37', 's39', 'f3n@', 'f3w@', 'm3', 'f1n@', 'b1d@', 'b2n#', 'f2d@', 'f1n#', 'b1w#', 's#', 'b1n#', 'x@', 'f1w#', 'y3', 'f2d#', 'b2d@', 'f2n#', 'f3d@', 'y1', 'b3w#', 'o1', 'f-3']


,match_id,Player1,Player2,Shots,Shots1,Shots0,Outcome,Server,Winner
0,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,4 b;37 f1 f1,b;37 f1,4 f1,*,0,1
1,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,6 b3,b3,6,w#,0,0
2,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,6,,6,*,0,0
3,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,5 b18 b2 f1 b2 f1 s2 b+3,b18 f1 f1 b+3,5 b2 b2 s2,*,0,1
4,20171015-M-Shanghai_Masters-F-Rafael_Nadal-Rog...,Rafael_Nadal,Roger_Federer,4 b27 f+3 b1,b27 b1,4 f+3,*,0,1


## Utils
Function to test the strategy, just provide the strategy

In [48]:
Model, shots_opo_com = aux_fn(df, poss_prompts, all_shots)


In [50]:
# Building up the model function

#################### The threshold on shots rarity ###################
threshold = 1                                                        #
######################################################################

# Given current prompt p_t and shot s_t, return next prompt p_(t+1)
def model(prompt, shot):
    n_grams_ps = Model[poss_prompts.index(prompt), all_shots.index(shot)]
    if(len(n_grams_ps) > threshold):    # exclude rare reactions
        return np.random.choice(n_grams_ps)
    return 'n@'   # when (p, s) never happened, return unforced error in net

In [27]:
# Test the strategy and returns the performance
def test_strategy(example_policy, number_of_points_played=150000):
    serves_opponent = [shot for shot in [sh[0] for sh in shots_opo_com] if shot[0] in '456']

    points_won  = 0
    points_lost = 0


    for num in range(number_of_points_played):
        print(num, end = '\r')
        # Random choice on who is serving
        if np.random.randint(0,2) == 0 :
            current_prompt = 'X'
        else :
            current_prompt = np.random.choice(serves_opponent)
        while True:
            shot = np.random.choice(example_policy.get(current_prompt,['5']))
            next_prompt = model(current_prompt, shot)
            current_prompt = next_prompt
            if is_win(current_prompt) == 1:
                points_won  += 1
                break
            if is_win(current_prompt) == 0:
                points_lost += 1
                break

    print('How well are we performing? '     , points_won/(points_lost + points_won))
    print('Baseline historical performance: ', 1 - (df['Winner'].sum()/len(df['Winner'])))
    return points_won/(points_lost + points_won)

## Data processing

Generates a dataframe with pair of shots (Nadal/Federer), the outcome and who won the points.



In [28]:
#create the table that we said at the beginning

#df #['Shots']
#df['Server']
#for i in df['Shots', 'Server']
#    print(i)
#df['Shots', 'Server']
#df_i = df[['Shots','Server']]
all_shots = []
nadal = {}
for index,row in df.iterrows():
    server = row.Server
    outcome = row.Outcome
    winner = row.Winner
    shots = row.Shots.split(' ')
    if server == 1:  #Federer
        shots = ['x'] + shots
        
    for i in range(0,len(shots),2):
        try:
            all_shots.append([shots[i], shots[i+1], outcome, winner])
        except:
            all_shots.append([shots[i], 'Z', outcome, winner])

all_shots_df = pd.DataFrame(all_shots, columns=['Nadal','Federer', 'Outcome', 'Winner'])
all_shots_df.head()

,Nadal,Federer,Outcome,Winner
0,4,b;37,*,1
1,f1,f1,*,1
2,6,b3,w#,0
3,6,Z,*,0
4,5,b18,*,1


## Testing the strategy

### Naive strategy

Simple policy, for each type Nadal shots just fill an array with Federer response.

__Outcome__: Federer wins only 10% of the time :(

### Better strategy

- Filter the data and keep only the points won by Federer
- Keep only the last pair of the exchange to build the strategy using the distribution of the responses

__Outcome__: winning 55%

In [45]:
def build_federer_winning_df():
    df_f = df[df.Winner==1]

    all_lasts = []
    for index, row in df_f.iterrows():
        server = row.Server
        shots = row.Shots.split(' ')
        outcome = row.Outcome
        #print(server, shots, outcome)
        if outcome in ['*', 'w*']:  # Last shot federer
            if len(shots) >= 2:
                last_nadal = shots[-2]
                last_federer = shots[-1]
            else:
                #print(server, shots, outcome)
                continue
        else:
            if len(shots) >= 3:
                last_nadal = shots[-3]
                last_federer = shots[-2]
            else:
                #print(server, shots, outcome)
                continue
        all_lasts.append([last_nadal, last_federer])
    all_lasts_df = pd.DataFrame(all_lasts, columns=['Nadal', 'Federer'])

    return all_lasts_df

def build_nadal_winning_df():
    df_f = df[df.Winner==0]

    all_lasts = []
    for index, row in df_f.iterrows():
        server = row.Server
        shots = row.Shots.split(' ')
        outcome = row.Outcome
        #print(server, shots, outcome)
        if outcome in ['*', 'w*']:  # Last shot nadal
            if len(shots) >= 2:
                last_nadal = shots[-1]
                last_federer = shots[-2]
            else:
                #print(server, shots, outcome)
                continue
        else:
            if len(shots) >= 3:
                last_nadal = shots[-2]
                last_federer = shots[-3]
            else:
                #print(server, shots, outcome)
                continue
        all_lasts.append([last_nadal, last_federer])
    all_lasts_df = pd.DataFrame(all_lasts, columns=['Nadal', 'Federer'])

    return all_lasts_df

        
def build_federer_strategy(all_lasts_df):
    fed_strat = {}
    fed_strat_prob = {}
    for index, row in all_lasts_df.iterrows():
        key = row['Nadal']
        value = row['Federer']
        if value not in all_shots:
            continue            
        if key not in fed_strat.keys():
            fed_strat[key] = [value]
        else:
            fed_strat[key].append(value)
    
    # add needed prompts
    for i in poss_prompts:
        if i not in fed_strat.keys():
            if i[-1] in ['@','#','*']:
                fed_strat[i] = ['']
            elif i == 'X':   #Nadal waits for us to serve
                fed_strat[i] = ['6']
            else:
                fed_strat[i] = ['f1']
    
    for i in fed_strat.keys():
        #fed_strat_prob[i] = 
        aux = dict(Counter(fed_strat[i]))
        
        for j in aux.keys():
            aux[j] = round(aux[j] / len(fed_strat[i]),2)
        fed_strat_prob[i] = aux
    
    return fed_strat, fed_strat_prob

def build_nadal_strategy(all_lasts_df):
    nad_strat = {}
    nad_strat_prob = {}
    for index, row in all_lasts_df.iterrows():
        key = row['Federer']
        value = row['Nadal']
        if value not in all_shots:
            continue            
        if key not in nad_strat.keys():
            nad_strat[key] = [value]
        else:
            nad_strat[key].append(value)
    
    # add needed prompts
    for i in poss_prompts:
        if i not in nad_strat.keys():
            if i[-1] in ['@','#','*']:
                nad_strat[i] = ['']
            elif i == 'X':   #Nadal waits for us to serve
                nad_strat[i] = ['6']
            else:
                nad_strat[i] = ['f1']
    
    for i in nad_strat.keys():
        #fed_strat_prob[i] = 
        aux = dict(Counter(nad_strat[i]))
        
        for j in aux.keys():
            aux[j] = round(aux[j] / len(nad_strat[i]),2)
        nad_strat_prob[i] = aux
    
    return nad_strat, nad_strat_prob

federer_df = build_federer_winning_df()
nadal_df = build_nadal_winning_df()

#print(fed_strat)
#fed_strat_prob
#nadal_df


federer_df.Nadal = federer_df.Nadal.str.strip('c')
federer_df.Federer = federer_df.Federer.str.strip('c')
nadal_df.Nadal = nadal_df.Nadal.str.strip('c')
nadal_df.Federer = nadal_df.Federer.str.strip('c')


fed_strat, fed_strat_prob = build_federer_strategy(federer_df)
nad_strat, nad_strat_prob = build_nadal_strategy(nadal_df)


In [51]:
test_strategy(fed_strat)

How well are we performing?  0.5553466666666667
Baseline historical performance:  0.5030959752321982


0.5553466666666667

### Best strategy

- Last pairs for both Nadal and Federer
- Reduce the weights  to the pairs giving the point to Federer

In [52]:
combined = {}
for i in poss_prompts:
    combined[i] = {}
    for j in fed_strat_prob[i].keys():
        combined[i][j] = fed_strat_prob[i][j] - 2.0*(len(nadal_df[nadal_df.Nadal==j]) / len(nadal_df))
        

for i in poss_prompts:
    for j in combined[i].keys():
        if combined[i][j] < 0:
            combined[i][j] = 0
for i in poss_prompts:
    for j in combined[i].keys():

        #combined[i][j] /= sum(combined[i].values())
        combined[i][j] *= 100

        combined2 = {}
        
for i in combined.keys():
    combined2[i] = []
    for j in combined[i].keys():
        combined2[i].extend([j]*int(combined[i][j]))

{'f3': ['b3',
  'b3',
  'b3',
  'b3',
  'b3',
  'b3',
  'b3',
  'b3',
  'b3',
  'b3',
  'b3',
  'b3',
  'b3',
  's2',
  's2',
  's2',
  'z1',
  'z1',
  'z1',
  'z1',
  's3',
  's3',
  's3',
  's3',
  's3',
  's3',
  's3',
  's3',
  's3',
  's3',
  's1',
  's1',
  's1',
  's1',
  'z3'],
 'f1': ['f1',
  'f1',
  'f1',
  'f1',
  'f1',
  'v1',
  'v1',
  'v1',
  'v1',
  'v1',
  'v1',
  'v1',
  'v1',
  'v1',
  'v1',
  'v1',
  'v1',
  'v1',
  'v1',
  'v1',
  'v1',
  'v1',
  'v1',
  'f2'],
 'b2': ['f+1',
  'f+1',
  'f+1',
  'f+1',
  'f+1',
  'f+1',
  'f+3',
  'f+3',
  'f+3',
  'f+3',
  'f+3',
  'f+3',
  'f+3',
  'f+3',
  'z3',
  'z3',
  'z3',
  'z3',
  'v3',
  'v3',
  'v3',
  'v1',
  'z1'],
 'b1': ['f1',
  'f1',
  'f1',
  'f1',
  'f1',
  'v3',
  'v3',
  'v3',
  'v3',
  'v3',
  'v3',
  'v3',
  'v3',
  'v3',
  'v3',
  'v3',
  'v1',
  'v1',
  'f+3'],
 'X': ['6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6',
  '6

In [53]:
test_strategy(combined2)

How well are we performing?  0.5981466666666667
Baseline historical performance:  0.5030959752321982


0.5981466666666667